# Deep Learning

In this Notebook, we will explore several deep learning techniques to solve text classification problems. 




- LSTM
- LSTM start with word2vec
- CNN
- LSTM + Attention https://blog.floydhub.com/attention-mechanism/


Convolutional Neural Networks (CNNs) have been shown to learn local features from words
or phrases [1], while Recurrent Neural Networks (RNNs) are able to learn temporal
dependencies in sequential data [2]. 

TODO
- fix warning
- try @tf.
- 

https://www.aclweb.org/anthology/O18-1021.pdf
    
    
https://arxiv.org/pdf/1702.01923.pdf

In [2]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_tensor_subword_8k, download_tfds_imdb_as_text_tiny, download_tfds_imdb_as_text
from lib.lstm import run_lstm_pipeline, run_cnn_pipeline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dataset = download_tfds_imdb_as_text()
dataset_tiny = download_tfds_imdb_as_text_tiny()

# Basic LSTM

We will try two slightly different uses of LSTM in text classification

- BiLSTMLastStateClassification - We use the Bi-directional LSTM to encode a vector that represent the meaning of the whole sentence, then feed it to two layers of fully connected neural networks to make a prediction. This model is very different from what we have done in Notebook 1 - 3 in a sense that it takes "order" in to account (all previous experiments are bag-of-word). For example, it can differentiate this classic example `This movie is boring it is not good` and `This movie is good it is not boring`. Although bigrams can differentiate this particular example, in the real world there are many other examples that bigram is not enough and you need 3-gram, 4-gram. Increasing n-gram is not the solution because it introduced sparseness to feature space. That's why LSTM comes in. The drawback of this model is that if the sentence is very long, the last output state may encode very little information of the very first tokens of the sentence. 

- BiLSTMPoolClassification - This is another use of LSTM. Instead of using the output of the last state of LSTM, now we use the LSTM output of every token in the sentence and, sum them up, and feed to two layers of FC layer. Since we sum up to output vector of each token, we go back to the bag-of-word approach again. However, it's not "very" bag-of-word. This approach is different from other bag-of-word style we used previously e.g. the Word2Vec bag-of-word in Notebook 2 - 3. In those models, we the word representing each tokens are static i.e. the vector for `good` are always same regarding where it presents in the sentence. However, in this architecture, the vector representing each token is contextualized It is aware of the word nearby because it's the output of LSTM. It can learn that vector for `good` can be different in different context. Moreover, since we use every LSTM output of each token, they also include the last state output as we used in above model. Although this vector is corresponding to the last token, it also represents the meaning of the whole sentence.

The  BiLSTMLastStateClassification is very similar to the TensorFlow official tutorial [here](https://www.tensorflow.org/tutorials/text/text_generation). However, we use the words as features to streamline with other experiments from Notebook 1 -4 while the tutorial use subwords. The model used in that tutorial has the F1 of 0.87.

In [ ]:
run_lstm_pipeline(dataset, "BiLSTMLastStateClassification", embeddings_size=64, hidden_unit=64)


In [ ]:
run_lstm_pipeline(dataset, "BiLSTMPoolClassification", embeddings_size=64, hidden_unit=64)


Discussion

# Basic LSTM + Pre-trained Word2Vec

In this experiment, we will see if it helps if we initialize the embeddings with pre-trained Word2Vec.

# CNN

CNN is widely used in Computer Vision. However, recent NLP researchers also adopt CV for many different NLP tasks. The CNN is different from LSTM in the sense that it examines few tokens within window size (5 in our experiment) then move to the next window. Then it pools (taking average in our experiment) the output from each windows to the single vector. Intuitively, the CNN should be able to capture the local semantic within the window size better.


This [paper](https://arxiv.org/pdf/1702.01923.pdf) discuss the comparative study of CNN and RNN for NLP tasks. 

In [ ]:
run_cnn_pipeline(dataset, embeddings_size=300, hidden_unit=64, window=5)

Epoch 1, Loss: 0.68, Accuracy: 0.62, Test Loss: 0.66, Test Accuracy: 0.74, Time: 170.20 s
